In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv
import re
import functools #for reduce_concat function
import datetime #for csv file title
import random

now = datetime.datetime.now() #for csv file title

In [2]:
#Initiate Chrome Driver and ignore certificate errors
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--ignore-ssl-errors')
driver = webdriver.Chrome(chrome_options=options)
driver.get("https://www.axs.com/browse/music")

In [4]:
#give the website time to load
time.sleep(5)
driver.maximize_window()

#Wait for JavaScript to appear before entering location
enterloc = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, "//body")))
time.sleep(5)

#Navigate to input for location and enter city name
enterloc = driver.find_element_by_xpath('//div[@class="nav-item-container"]/ul/li/a[@class="nav-location location-search-trigger"]')
enterloc.click()
time.sleep(1.5)
typeloc = driver.find_element_by_xpath('//div/input[@class="location-search__form-input"]')

#Type the string more slowly to avoid detection
keysender = "New York City, NY"
keysender.split()
for i in keysender:
	typeloc.send_keys(i)
	time.sleep(0.1)
	typeloc.send_keys(Keys.END)
	time.sleep(0.1)
time.sleep(2)
typeloc.send_keys(Keys.ENTER)
time.sleep(3)

In [196]:
#scroll to end of page so that all elements are visible
calm = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, "//html"))) #if elements aren't available it will return a timeout error and alert user that script must be altered
time.sleep(3)
lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
match=False

while(match==False):
	lastCount = lenOfPage
	time.sleep(3.5)
	lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
	if lastCount==lenOfPage:
		match=True

scroll = driver.find_element_by_xpath('//html')
time.sleep(2)
scroll.send_keys(Keys.HOME)
time.sleep(2)

In [236]:
#Open CSV Writer
filename = paste(['axs_events_'], [now.year], ['-'], [now.month], ['-'], [now.day],['.csv'], sep='')
filename = ''.join(filename)
csv_file = open(filename, 'a', newline = '')
writer = csv.writer(csv_file)

#collect html tags, this will need to go inside of the nested loop function
wait_event = WebDriverWait(driver, 10)
events = wait_event.until(EC.presence_of_all_elements_located((By.XPATH,
							'//div[@class="results-table results-table--events"]')))

In [232]:
events[-1].text

'FEB\n24\nSAT\nFrightened Rabbit: The Midnight Organ Fight 10th Anniversary Tour\nMusic Hall of Williamsburg, Brooklyn NY'

In [213]:
len(events)

53

In [149]:
print(lenOfPage)

2283


In [30]:
href = hold.get_attribute('href')
print(href)

http://www.axs.com/events/344193/first-aid-kit-tickets


In [191]:
link = events[2].find_element_by_xpath('./div/a')
sale_url = link.get_attribute('href')
month = events[2].find_element_by_xpath('./div/a/div[1]/div/div[1]').text
day = events[2].find_element_by_xpath('./div/a/div[1]/div/div[2]').text
weekday = events[2].find_element_by_xpath('./div/a/div[1]/div/div[3]').text
performer = events[2].find_element_by_xpath('./div/a/div[2]/div[1]').text
venue = events[2].find_element_by_xpath('./div/a/div[2]/div[2]/span[1]').text
city = events[2].find_element_by_xpath('./div/a/div[2]/div[2]/span[2]').text

In [192]:
performer

'Sleigh Bells'

In [193]:
driver.get(sale_url)
new_page_wait = wait_event.until(EC.presence_of_all_elements_located((By.XPATH,
                    '//body')))
hour = driver.find_element_by_xpath('//section[@id="event-info-section"]/div[2]/div[2]/span[1]').text
price = driver.find_element_by_xpath('//section[@id="event-ticket-options"]/div/div[2]/div').text
time.sleep(2)
driver.back()

In [195]:
price

'Price: $30.00'

In [516]:
#For csv file names (testing)
def reduce_concat(x, sep=""):
	return functools.reduce(lambda x, y: str(x) + sep + str(y), x)
	
def paste(*lists, sep=" ", collapse=None):
	result = map(lambda x: reduce_concat(x, sep=sep), zip(*lists))
	if collapse is not None:
		return reduce_concat(result, sep=collapse)
	return list(result)
	
def writeevents(events, writer, driver):
	for i in range(5, len(events)):
		#dictionary to store event data
		event_dict = {}
		
		#set event data equal to html elements
		link = events[i].find_element_by_xpath('./div/a')
		sale_url = link.get_attribute('href')
		month = events[i].find_element_by_xpath('./div/a/div[1]/div/div[1]').text
		day = events[i].find_element_by_xpath('./div/a/div[1]/div/div[2]').text
		weekday = events[i].find_element_by_xpath('./div/a/div[1]/div/div[3]').text
		performer = events[i].find_element_by_xpath('./div/a/div[2]/div[1]').text
		venue = events[i].find_element_by_xpath('./div/a/div[2]/div[2]/span[1]').text
		city = events[i].find_element_by_xpath('./div/a/div[2]/div[2]/span[2]').text
		time.sleep(random.uniform(0.4, 0.6))
		
		#navigate to a new page for time and price
		driver.get(sale_url)
		new_page_wait = wait_event.until(EC.presence_of_all_elements_located((By.XPATH,
							'//body')))
		hour = driver.find_element_by_xpath('//section[@id="event-info-section"]/div[2]/div[2]/span[1]').text
		price = driver.find_element_by_xpath('//section[@id="event-ticket-options"]/div/div[2]/div').text
		time.sleep(2)
		driver.back()
		
		#write variables to dictionary
		event_dict['sale_url'] = sale_url
		event_dict['month'] = month
		event_dict['day'] = day
		event_dict['weekday'] = weekday
		event_dict['performer'] = performer
		event_dict['venue'] = venue
		event_dict['city'] = city
		event_dict['time'] = hour
		event_dict['price'] = price
		
		#ensure accurate data collection
		print(i)
		i
		print(event_dict)
		event_dict
		
		#write event to CSV
		writer.writerow(event_dict.values())
		
		calm = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, "//html"))) #if elements aren't available it will return a timeout error and alert user that script must be altered
		time.sleep(3)
		lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
		match=False

		while(match==False):
			lastCount = lenOfPage
			time.sleep(random.uniform(3.5,5))
			lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
			if lastCount==lenOfPage:
				match=True
                
		scrollerArg = paste(["window.scrollTo(0, "], [5724 * 5.5], [");"], sep = '')
		scrollerArg = ''.join(scrollerArg)
		scroller = driver.execute_script(scrollerArg)
		
		scrollerArg = paste(["window.scrollTo(0, "], [newPos], [");"], sep = '')
		scrollerArg = ''.join(scrollerArg)
		scroller = driver.execute_script(scrollerArg)
		events = wait_event.until(EC.presence_of_all_elements_located((By.XPATH,
					'//div[@class="results-table results-table--events"]')))

In [519]:
scrollerArg = paste(["window.scrollTo(0, "], [5724*5.5], [");"], sep = '')
scrollerArg = ''.join(scrollerArg)
scroller = driver.execute_script(scrollerArg)
position = 5724*5.5
print(position)

31482.0


In [508]:
5724 * 5.5

31482.0

In [520]:
#Open CSV Writer
filename = paste(['axs_events_'], [now.year], ['-'], [now.month], ['-'], [now.day],['.csv'], sep='')
filename = ''.join(filename)
csv_file = open(filename, 'a', newline = '')
writer = csv.writer(csv_file)

#collect html tags, this will need to go inside of the nested loop function
wait_event = WebDriverWait(driver, 10)
events = wait_event.until(EC.presence_of_all_elements_located((By.XPATH,
							'//div[@class="results-table results-table--events"]')))

In [521]:
newPos = 5724 * 5.5
while (newPos <= lenOfPage):
	print('Number of events:', len(events))
	print('position pre-loop:', newPos) #for testing
	writeevents(events, writer, driver)
	offprep = paste(["var offprep = window.screen.height * "], [(len(events))/8], [";return offprep"], sep='') #7 fit on a page, giving myself room for error
	offprep = ''.join(offprep)
	offset = driver.execute_script(offprep)
	newPos = newPos + offset
	scrollerArg = paste(["window.scrollTo(0, "], [newPos], [");"], sep = '')
	scrollerArg = ''.join(scrollerArg)
	scroller = driver.execute_script(scrollerArg)
	print(newPos)
	events = wait_event.until(EC.presence_of_all_elements_located((By.XPATH,
				'//div[@class="results-table results-table--events"]')))
	print('Number of events:', len(events)) #testing
	print('current position:', newPos) #testing
print('Complete.')
csv_file.close()
driver.close()

Number of events: 75
position pre-loop: 31482.0
5
{'sale_url': 'http://www.axs.com/events/348539/bishop-briggs-tickets', 'month': 'MAY', 'day': '22', 'weekday': 'TUE', 'performer': 'Bishop Briggs', 'venue': 'Brooklyn Steel,', 'city': 'Brooklyn NY', 'time': 'Brooklyn, NY - 8:00 PM', 'price': 'Price: $23.00 - $73.00'}
6
{'sale_url': 'http://www.axs.com/events/336284/the-kooks-tickets', 'month': 'MAY', 'day': '23', 'weekday': 'WED', 'performer': 'The Kooks', 'venue': 'Brooklyn Steel,', 'city': 'Brooklyn NY', 'time': 'Brooklyn, NY - 8:00 PM', 'price': 'Price: $35.00'}
7
{'sale_url': 'http://www.axs.com/events/347845/billy-joel-in-concert-tickets', 'month': 'MAY', 'day': '23', 'weekday': 'WED', 'performer': 'Billy Joel - In Concert', 'venue': 'Madison Square Garden,', 'city': 'New York City NY', 'time': 'New York City, NY - 8:00 PM', 'price': ''}
8
{'sale_url': 'http://www.axs.com/events/347861/this-is-the-kit-tickets', 'month': 'MAY', 'day': '23', 'weekday': 'WED', 'performer': 'This Is th

KeyboardInterrupt: 

In [522]:
csv_file.close()

In [507]:
!type axs_events_2018-2-12.csv

http://www.axs.com/events/348868/mgmt-tickets,MAR,26,MON,MGMT,"Brooklyn Steel,",Brooklyn NY,"Brooklyn, NY - 8:00 PM",Price: $45.00
http://www.axs.com/events/346097/rob-bell-tickets,MAR,26,MON,Rob Bell,"Music Hall of Williamsburg,",Brooklyn NY,"Brooklyn, NY - 8:30 PM",Price: $35.00 - $60.00
http://www.axs.com/events/347896/dead-meadow-tickets,MAR,26,MON,Dead Meadow,"Rough Trade NYC,",Brooklyn NY,"Brooklyn, NY - 8:30 PM",Price: $18.00
http://www.axs.com/events/345171/dan-auerbach-the-easy-eye-sound-revue-featuring-robert-finley-shannon-tickets,MAR,27,TUE,"Dan Auerbach & The Easy Eye Sound Revue featuring Robert Finley, Shannon Shaw","Brooklyn Steel,",Brooklyn NY,"Brooklyn, NY - 8:00 PM",Price: $45.00
http://www.axs.com/events/344133/the-strypes-tickets,MAR,27,TUE,The Strypes,"Rough Trade NYC,",Brooklyn NY,"Brooklyn, NY - 9:00 PM",Price: $15.00
http://www.axs.com/events/345955/marlon-williams-tickets,MAR,27,TUE,Marlon Williams,"Music Hall of Williamsburg,",Brooklyn NY,"Brooklyn, NY - 9:00

In [253]:
		event_dict = {}
		
		link = events[19].find_element_by_xpath('./div/a')
		sale_url = link.get_attribute('href')
		month = events[19].find_element_by_xpath('./div/a/div[1]/div/div[1]').text
		day = events[19].find_element_by_xpath('./div/a/div[1]/div/div[2]').text
		weekday = events[19].find_element_by_xpath('./div/a/div[1]/div/div[3]').text
		performer = events[19].find_element_by_xpath('./div/a/div[2]/div[1]').text
		venue = events[19].find_element_by_xpath('./div/a/div[2]/div[2]/span[1]').text
		city = events[19].find_element_by_xpath('./div/a/div[2]/div[2]/span[2]').text
		time.sleep(0.5)
		
		#navigate to a new page for time and price
		driver.get(sale_url)
		new_page_wait = wait_event.until(EC.presence_of_all_elements_located((By.XPATH,
							'//body')))
		hour = driver.find_element_by_xpath('//section[@id="event-info-section"]/div[2]/div[2]/span[1]').text
		price = driver.find_element_by_xpath('//section[@id="event-ticket-options"]/div/div[2]/div').text
		time.sleep(2)
		driver.back()
		
		#write variables to dictionary
		event_dict['sale_url'] = sale_url
		event_dict['month'] = month
		event_dict['day'] = day
		event_dict['weekday'] = weekday
		event_dict['performer'] = performer
		event_dict['venue'] = venue
		event_dict['city'] = city
		event_dict['time'] = hour
		event_dict['price'] = price
		print(event_dict)
		event_dict
		
		#write event to CSV
		writer.writerow(event_dict.values())

{'sale_url': 'http://www.axs.com/events/346741/railroad-earth-tickets', 'month': 'FEB', 'day': '17', 'weekday': 'SAT', 'performer': 'Railroad Earth', 'venue': 'The Capitol Theatre,', 'city': 'Port Chester NY', 'time': 'Port Chester, NY - 8:00 PM', 'price': 'Price: $25.00 - $35.00'}


174

In [515]:
events[5].text

'MAY\n22\nTUE\nBishop Briggs\nBrooklyn Steel, Brooklyn NY'

In [517]:
len(events)

75